# AutoDL - Regression

## Imports and Global Settings

In [1]:
import os
import sys
import datetime
import json
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import tensorflow as tf
import autokeras as ak
from sklearn.metrics import mean_absolute_error, r2_score

here = os.getcwd()
sys.path.append(os.path.join(here, ".."))

from src.utils.modeling_utils import (
    ModelSetup,
    evaluate_reg_model,
    calculate_roi,
    save_model_report,
)

load_dotenv()
RDS_ENDPOINT = os.getenv("RDS_ENDPOINT")
RDS_PASSWORD = os.getenv("RDS_PASSWORD")

# Pandas Settings
pd.set_option("display.max_columns", 1000)
pd.set_option("display.max_rows", 1000)
pd.options.display.max_info_columns = 200
pd.options.display.precision = 5

2023-09-20 09:19:59.106744: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 09:19:59.161573: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 09:19:59.163051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 09:20:00.238943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


## Load Data

In [2]:
username = "postgres"
password = RDS_PASSWORD
endpoint = RDS_ENDPOINT
database = "nba_betting"
port = "5432"

# Create the connection string
connection_string = (
    f"postgresql+psycopg2://{username}:{password}@{endpoint}:{port}/{database}"
)

### Games

In [3]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = [
    "game_id",
    "game_datetime",
    "home_team",
    "away_team",
    "open_line",
    "home_score",
    "away_score",
    "game_completed",
    "odds_last_update",
    "scores_last_update",
]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
games_query = f"SELECT {', '.join(features)} FROM games WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    games = pd.read_sql_query(games_query, connection)

### Features

In [4]:
start_date = "2020-09-01"
start_date_int = int(start_date.replace("-", ""))  # Convert date to YYYYMMDD format

features = ["game_id", "data"]

# Extracting the YYYYMMDD portion of the game_id and comparing it with start_date_int
features_query = f"SELECT {', '.join(features)} FROM all_features_json WHERE CAST(LEFT(game_id, 8) AS INTEGER) >= {start_date_int};"

with create_engine(connection_string).connect() as connection:
    all_features = pd.read_sql_query(features_query, connection)

# Normalize the JSON strings in the 'data' column
expanded_data = pd.json_normalize(all_features["data"])

# Drop the original 'data' column and concatenate the expanded data
all_features = pd.concat([all_features.drop(columns=["data"]), expanded_data], axis=1)

In [5]:
games_features = pd.merge(
    games,
    all_features,
    on="game_id",
    how="left",
    validate="one_to_one",
    suffixes=("", "_drop"),
)
# Drop the columns from df2 (with suffix '_drop')
games_features = games_features[
    games_features.columns.drop(list(games_features.filter(regex="_drop")))
]

<a id='basic_data_overview'></a>

## Basic Data Overview

In [6]:
df = games_features.copy()

In [7]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3956 entries, 0 to 3955
Data columns (total 778 columns):
 #    Column                                        Non-Null Count  Dtype         
---   ------                                        --------------  -----         
 0    game_id                                       3956 non-null   object        
 1    game_datetime                                 3956 non-null   datetime64[ns]
 2    home_team                                     3956 non-null   object        
 3    away_team                                     3956 non-null   object        
 4    open_line                                     3853 non-null   float64       
 5    home_score                                    3855 non-null   float64       
 6    away_score                                    3855 non-null   float64       
 7    game_completed                                3956 non-null   bool          
 8    odds_last_update                              94 non-nul

In [8]:
df.head(10)

,game_id,game_datetime,home_team,away_team,open_line,home_score,away_score,game_completed,odds_last_update,scores_last_update,season,away_ATL,away_BKN,away_BOS,away_CHA,away_CHI,away_CLE,away_DAL,away_DEN,away_DET,away_GSW,away_HOU,away_IND,away_LAC,away_LAL,away_MEM,away_MIA,away_MIL,away_MIN,away_NOP,away_NYK,away_OKC,away_ORL,away_PHI,away_PHX,away_POR,away_SAC,away_SAS,away_TOR,away_UTA,away_WAS,elo1_pre,elo2_pre,home_ATL,home_BKN,home_BOS,home_CHA,home_CHI,home_CLE,home_DAL,home_DEN,home_DET,home_GSW,home_HOU,home_IND,home_LAC,home_LAL,home_MEM,home_MIA,home_MIL,home_MIN,home_NOP,home_NYK,home_OKC,home_ORL,home_PHI,home_PHX,home_POR,home_SAC,home_SAS,home_TOR,home_UTA,home_WAS,538_prob1,elo_prob1,elo_prob2,last_5_hv,streak_hv,win_pct_hv,raptor1_pre,raptor2_pre,season_type,raptor_prob1,raptor_prob2,rest_diff_hv,carm_elo1_pre,carm_elo2_pre,day_of_season,point_diff_hv,carm_elo_prob1,carm_elo_prob2,away_team_last_5,away_team_streak,home_team_last_5,home_team_streak,away_team_win_pct,home_team_win_pct,point_diff_last_5_hv,pie_away_all_advanced,pie_away_l2w_advanced,pie_home_all_advanced,pie_home_l2w_advanced,l_away_all_traditional,l_away_l2w_traditional,l_home_all_traditional,l_home_l2w_traditional,pace_away_all_advanced,pace_away_l2w_advanced,pace_home_all_advanced,pace_home_l2w_advanced,poss_away_all_advanced,poss_away_l2w_advanced,poss_home_all_advanced,poss_home_l2w_advanced,w_away_all_traditional,w_away_l2w_traditional,w_home_all_traditional,w_home_l2w_traditional,gp_away_all_traditional,gp_away_l2w_traditional,gp_home_all_traditional,gp_home_l2w_traditional,ast_to_away_all_advanced,ast_to_away_l2w_advanced,ast_to_home_all_advanced,ast_to_home_l2w_advanced,away_team_avg_point_diff,e_pace_away_all_advanced,e_pace_away_l2w_advanced,e_pace_home_all_advanced,e_pace_home_l2w_advanced,home_team_avg_point_diff,min_away_all_traditional,min_away_l2w_traditional,min_home_all_traditional,min_home_l2w_traditional,opp_pf_away_all_opponent,opp_pf_away_l2w_opponent,opp_pf_home_all_opponent,opp_pf_home_l2w_opponent,pts_away_all_traditional,pts_away_l2w_traditional,pts_home_all_traditional,pts_home_l2w_traditional,ts_pct_away_all_advanced,ts_pct_away_l2w_advanced,ts_pct_home_all_advanced,ts_pct_home_l2w_advanced,ast_pct_away_all_advanced,ast_pct_away_l2w_advanced,ast_pct_home_all_advanced,ast_pct_home_l2w_advanced,days_since_last_game_away,days_since_last_game_home,efg_pct_away_all_advanced,efg_pct_away_l2w_advanced,efg_pct_home_all_advanced,efg_pct_home_l2w_advanced,opp_ast_away_all_opponent,opp_ast_away_l2w_opponent,opp_ast_home_all_opponent,opp_ast_home_l2w_opponent,opp_blk_away_all_opponent,opp_blk_away_l2w_opponent,opp_blk_home_all_opponent,opp_blk_home_l2w_opponent,opp_fga_away_all_opponent,opp_fga_away_l2w_opponent,opp_fga_home_all_opponent,opp_fga_home_l2w_opponent,opp_fgm_away_all_opponent,opp_fgm_away_l2w_opponent,opp_fgm_home_all_opponent,opp_fgm_home_l2w_opponent,opp_fta_away_all_opponent,opp_fta_away_l2w_opponent,opp_fta_home_all_opponent,opp_fta_home_l2w_opponent,opp_ftm_away_all_opponent,opp_ftm_away_l2w_opponent,opp_ftm_home_all_opponent,opp_ftm_home_l2w_opponent,opp_pfd_away_all_opponent,opp_pfd_away_l2w_opponent,opp_pfd_home_all_opponent,opp_pfd_home_l2w_opponent,opp_pts_away_all_opponent,opp_pts_away_l2w_opponent,opp_pts_home_all_opponent,opp_pts_home_l2w_opponent,opp_reb_away_all_opponent,opp_reb_away_l2w_opponent,opp_reb_home_all_opponent,opp_reb_home_l2w_opponent,opp_stl_away_all_opponent,opp_stl_away_l2w_opponent,opp_stl_home_all_opponent,opp_stl_home_l2w_opponent,opp_tov_away_all_opponent,opp_tov_away_l2w_opponent,opp_tov_home_all_opponent,opp_tov_home_l2w_opponent,reb_pct_away_all_advanced,reb_pct_away_l2w_advanced,reb_pct_home_all_advanced,reb_pct_home_l2w_advanced,dreb_pct_away_all_advanced,dreb_pct_away_l2w_advanced,dreb_pct_home_all_advanced,dreb_pct_home_l2w_advanced,opp_blka_away_all_opponent,opp_blka_away_l2w_opponent,opp_blka_home_all_opponent,opp_blka_home_l2w_opponent,opp_dreb_away_all_opponent,opp_dreb_

## Data Preparation

#### Drop Non-Completed Games and Games with No Line

In [9]:
df = df[df["game_completed"] == True]
df = df.dropna(subset=["open_line"])

### Create Targets

In [10]:
df = ModelSetup.add_targets(df)

### Select Features

In [11]:
training_seasons = [x for x in range(2010, 2022)]
training_dates, testing_dates = ModelSetup.choose_dates(training_seasons, [2022], "Reg")
print("Training Dates:")
print(training_dates)
print("Testing Dates:")
print(testing_dates)

Training Dates:
('2010-10-26', '2022-04-10')
Testing Dates:
('2022-10-18', '2023-04-09')


In [12]:
for col in df.columns:
    print(col)

game_id
game_datetime
home_team
away_team
open_line
home_score
away_score
game_completed
odds_last_update
scores_last_update
season
away_ATL
away_BKN
away_BOS
away_CHA
away_CHI
away_CLE
away_DAL
away_DEN
away_DET
away_GSW
away_HOU
away_IND
away_LAC
away_LAL
away_MEM
away_MIA
away_MIL
away_MIN
away_NOP
away_NYK
away_OKC
away_ORL
away_PHI
away_PHX
away_POR
away_SAC
away_SAS
away_TOR
away_UTA
away_WAS
elo1_pre
elo2_pre
home_ATL
home_BKN
home_BOS
home_CHA
home_CHI
home_CLE
home_DAL
home_DEN
home_DET
home_GSW
home_HOU
home_IND
home_LAC
home_LAL
home_MEM
home_MIA
home_MIL
home_MIN
home_NOP
home_NYK
home_OKC
home_ORL
home_PHI
home_PHX
home_POR
home_SAC
home_SAS
home_TOR
home_UTA
home_WAS
538_prob1
elo_prob1
elo_prob2
last_5_hv
streak_hv
win_pct_hv
raptor1_pre
raptor2_pre
season_type
raptor_prob1
raptor_prob2
rest_diff_hv
carm_elo1_pre
carm_elo2_pre
day_of_season
point_diff_hv
carm_elo_prob1
carm_elo_prob2
away_team_last_5
away_team_streak
home_team_last_5
home_team_streak
away_team_win_pct
ho

In [13]:
features_to_use = [
    "open_line",
    "rest_diff_hv",
    "day_of_season",
    "last_5_hv",
    "538_prob1",
    "elo_prob1",
    "streak_hv",
    "point_diff_last_5_hv",
    "point_diff_hv",
    "win_pct_hv",
    "plus_minus_home_l2w_traditional",
    "net_rating_home_l2w_advanced",
    "plus_minus_home_l2w_opponent",
    "plus_minus_zscore_home_l2w_traditional",
    "net_rating_zscore_home_l2w_advanced",
    "plus_minus_zscore_home_l2w_opponent",
    "e_net_rating_home_l2w_advanced",
    "e_net_rating_zscore_home_l2w_advanced",
    "plus_minus_percentile_home_l2w_opponent",
    "plus_minus_percentile_home_l2w_traditional",
    "net_rating_percentile_home_l2w_advanced",
    "plus_minus_away_l2w_traditional",
    "plus_minus_away_l2w_opponent",
    "w_pct_zscore_home_l2w_traditional",
    "e_net_rating_percentile_home_l2w_advanced",
    "e_net_rating_away_l2w_advanced",
    "pie_percentile_home_l2w_advanced",
    "e_net_rating_zscore_away_l2w_advanced",
    "net_rating_zscore_away_l2w_advanced",
    "pie_home_l2w_advanced",
]

In [14]:
df.dropna(subset=features_to_use, inplace=True)

In [15]:
training_df, testing_df, model_report = ModelSetup.create_datasets(
    df, "reg", features_to_use, training_dates, testing_dates, create_report=True
)

In [16]:
print("Training Shape: ", training_df.shape)
print("Testing Shape: ", testing_df.shape)

Training Shape:  (2370, 33)
Testing Shape:  (1199, 33)


### Baselines

In [17]:
training_baseline_via_vegas = model_report["ind_baseline_train"]
testing_baseline_via_vegas = model_report["ind_baseline_test"]

training_baseline_via_mean = model_report["dep_baseline_train"]
testing_baseline_via_mean = model_report["dep_baseline_test"]

print(f"Training Baseline via Vegas: {training_baseline_via_vegas:.2f}")
print(f"Testing Baseline via Vegas: {testing_baseline_via_vegas:.2f}")
print(f"Training Baseline via Mean: {training_baseline_via_mean:.2f}")
print(f"Testing Baseline via Mean: {testing_baseline_via_mean:.2f}")

Training Baseline via Vegas: 10.65
Testing Baseline via Vegas: 9.73
Training Baseline via Mean: 12.12
Testing Baseline via Mean: 10.94


<a id=Regression></a>

## Regression

In [18]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2370 entries, 154 to 2595
Data columns (total 33 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   game_id                                     2370 non-null   object 
 1   vegas_open_hv                               2370 non-null   float64
 2   REG_TARGET                                  2370 non-null   float64
 3   open_line                                   2370 non-null   float64
 4   rest_diff_hv                                2370 non-null   float64
 5   day_of_season                               2370 non-null   float64
 6   last_5_hv                                   2370 non-null   float64
 7   538_prob1                                   2370 non-null   float64
 8   elo_prob1                                   2370 non-null   float64
 9   streak_hv                                   2370 non-null   float64
 10  point_diff

In [19]:
X_train = training_df[features_to_use]
X_test = testing_df[features_to_use]
y_train = training_df["REG_TARGET"]
y_test = testing_df["REG_TARGET"]

## AutoKeras

In [20]:
reg = ak.StructuredDataRegressor(
    max_trials=100,
    overwrite=True,
    loss="mean_absolute_error",
)

In [21]:
reg.fit(X_train, y_train)

Trial 75 Complete [00h 00m 08s]
val_loss: 7.81294584274292

Best val_loss So Far: 7.359921455383301
Total elapsed time: 00h 59m 10s
Epoch 1/126
75/75 [==============================] - 1s 2ms/step - loss: 10.8012 - mean_squared_error: 184.5459
Epoch 2/126
75/75 [==============================] - 0s 2ms/step - loss: 9.7840 - mean_squared_error: 154.4898
Epoch 3/126
75/75 [==============================] - 0s 3ms/step - loss: 9.5932 - mean_squared_error: 147.7537
Epoch 4/126
75/75 [==============================] - 0s 2ms/step - loss: 9.3165 - mean_squared_error: 140.1399
Epoch 5/126
75/75 [==============================] - 0s 2ms/step - loss: 9.1382 - mean_squared_error: 135.6702
Epoch 6/126
75/75 [==============================] - 0s 2ms/step - loss: 9.0824 - mean_squared_error: 133.4453
Epoch 7/126
75/75 [==============================] - 0s 2ms/step - loss: 8.8551 - mean_squared_error: 127.4813
Epoch 8/126
75/75 [==============================] - 0s 2ms/step - loss: 8.7514 - mean_squ

INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


In [22]:
# Evaluate the best model with testing data.
print(reg.evaluate(X_test, y_test))

38/38 [==============================] - 0s 3ms/step - loss: 6.5974 - mean_squared_error: 69.5311
[6.5973992347717285, 69.53105926513672]


In [23]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 multi_category_encoding (M  (None, 30)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 30)                61        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                992       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dropout (Dropout)           (None, 32)                0     

In [24]:
model_report["details"] = model.get_config()

<a id=evaluate></a>

### Evaluate Model

In [25]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

38/38 [==============================] - 0s 2ms/step


In [26]:
train_mae = mean_absolute_error(y_train, train_pred)
train_r2 = r2_score(y_train, train_pred)
print(f"Training MAE: {train_mae:.2f}")
print(f"Training R2: {train_r2:.2f}")

Training MAE: 6.80
Training R2: 0.67


In [27]:
model_report["train_mae"] = train_mae
model_report["train_r2"] = train_r2

In [28]:
test_mae = mean_absolute_error(y_test, test_pred)
test_r2 = r2_score(y_test, test_pred)
print(f"Testing MAE: {test_mae:.2f}")
print(f"Testing R2: {test_r2:.2f}")

Testing MAE: 6.60
Testing R2: 0.62


In [29]:
model_report["test_mae"] = test_mae
model_report["test_r2"] = test_r2

In [30]:
vegas_pred_train = -X_train["open_line"]
train_pred = train_pred.flatten()
train_pred_results = pd.DataFrame(
    {"vegas_open_hv": vegas_pred_train, "model_pred": train_pred, "actual": y_train}
)

In [31]:
vegas_pred_test = -X_test["open_line"]
test_pred = test_pred.flatten()
test_pred_results = pd.DataFrame(
    {"vegas_open_hv": vegas_pred_test, "model_pred": test_pred, "actual": y_test}
)

In [32]:
train_acc_reg, train_closer_to_target_reg, train_prediction_df_reg = evaluate_reg_model(
    train_pred_results, "vegas_open_hv", "actual", "model_pred"
)
test_acc_reg, test_closer_to_target_reg, test_prediction_df_reg = evaluate_reg_model(
    test_pred_results, "vegas_open_hv", "actual", "model_pred"
)

Prediction is closer to target in 69.16% of cases
Accuracy: 0.7886
Prediction is closer to target in 63.89% of cases
Accuracy: 0.7665


In [33]:
model_report["train_acc_reg"] = train_acc_reg
model_report["test_acc_reg"] = test_acc_reg
model_report["train_ctt"] = train_closer_to_target_reg
model_report["test_ctt"] = test_closer_to_target_reg

In [34]:
roi_results_reg = calculate_roi(test_prediction_df_reg, "actual_side", "pred_side")
roi_results_reg

,Label,Total ROI,Average ROI per Bet
0,"All Bets, Even Amount",63900,53.29
1,"All Bets, Typical Odds",55629,46.40


In [35]:
model_report["roi_all_bets_even_amount_avg"] = roi_results_reg[
    roi_results_reg["Label"] == "All Bets, Even Amount"
]["Average ROI per Bet"].iloc[0]
model_report["roi_all_bets_typical_odds_avg"] = roi_results_reg[
    roi_results_reg["Label"] == "All Bets, Typical Odds"
]["Average ROI per Bet"].iloc[0]

### Model Storage

In [36]:
platform = "autokeras"
problem_type = "reg"
model_type = "dl"
datetime_str = model_report["datetime"].strftime("%Y_%m_%d_%H_%M_%S")

model_id = f"{platform}_{problem_type}_{model_type}_{datetime_str}"
model_id

'autokeras_reg_dl_2023_09_20_09_20_15'

In [37]:
model.save(f"../models/AutoDL/{model_id}", save_format="tf")

INFO:tensorflow:Assets written to: ../models/AutoDL/autokeras_reg_dl_2023_09_20_09_20_15/assets


INFO:tensorflow:Assets written to: ../models/AutoDL/autokeras_reg_dl_2023_09_20_09_20_15/assets


In [38]:
model_report["platform"] = platform
model_report["model_type"] = model_type
model_report["model_id"] = model_id

In [39]:
model_report

{'datetime': Timestamp('2023-09-20 09:20:15.322515'),
 'problem_type': 'reg',
 'target': 'REG_TARGET',
 'features': ['open_line',
  'rest_diff_hv',
  'day_of_season',
  'last_5_hv',
  '538_prob1',
  'elo_prob1',
  'streak_hv',
  'point_diff_last_5_hv',
  'point_diff_hv',
  'win_pct_hv',
  'plus_minus_home_l2w_traditional',
  'net_rating_home_l2w_advanced',
  'plus_minus_home_l2w_opponent',
  'plus_minus_zscore_home_l2w_traditional',
  'net_rating_zscore_home_l2w_advanced',
  'plus_minus_zscore_home_l2w_opponent',
  'e_net_rating_home_l2w_advanced',
  'e_net_rating_zscore_home_l2w_advanced',
  'plus_minus_percentile_home_l2w_opponent',
  'plus_minus_percentile_home_l2w_traditional',
  'net_rating_percentile_home_l2w_advanced',
  'plus_minus_away_l2w_traditional',
  'plus_minus_away_l2w_opponent',
  'w_pct_zscore_home_l2w_traditional',
  'e_net_rating_percentile_home_l2w_advanced',
  'e_net_rating_away_l2w_advanced',
  'pie_percentile_home_l2w_advanced',
  'e_net_rating_zscore_away_l2w_a

In [40]:
save_model_report(model_report)

/home/jeff/Documents/NBA_Betting/notebooks/../src/utils/modeling_utils.py:447: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  updated_reports_df = existing_reports_df.append(new_report_df, ignore_index=True)
